In [1]:
# customizing_llm_local.py

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# 1. Load a lightweight model locally (you can change this to any model you have)
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

# 2. Define a prompt for fine-tuning / customization
prompt = """You are an assistant specialized in summarizing research papers.
Summarize the following abstract in one sentence:
'The paper introduces a new method for optimizing neural networks using evolutionary algorithms...'"""

# 3. Generate a customized response
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
output = pipe(prompt, max_new_tokens=100, temperature=0.7)

print("\n--- Customized LLM Output ---\n")
print(output[0]["generated_text"])


Loading model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



--- Customized LLM Output ---

You are an assistant specialized in summarizing research papers.
Summarize the following abstract in one sentence:
'The paper introduces a new method for optimizing neural networks using evolutionary algorithms...'

This paper proposes a new approach for enhancing the efficiency of neural networks through the application of evolutionary algorithms in optimization.

Reference(s):
Title: Optimizing Deep Neural Networks using Evolutionary Algorithms
Authors: Xinlei Chen, Ying Zhang, and Jianbo Shi
Publication: IEEE Transactions on Evolutionary Computation
Year: 2018
Volume: 22
Iss


In [5]:
pip install peft datasets


In [6]:
from peft import LoraConfig, get_peft_model

# Configure LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 3,407,872 || all params: 7,251,431,424 || trainable%: 0.0470
